# Question 6

# Data Normalization

In [20]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('./train_data.csv').to_numpy()
val_df = pd.read_csv('./validate_data.csv').to_numpy()

y_train=train_df[:,2]
x_train=train_df[:,0:2]
y_val=val_df[:,2]
x_val=val_df[:,0:2]



In [21]:
def minMaxNorm(data):
    return  (data - np.min(data)) /  (np.max(data) - np.min(data))

x_train= minMaxNorm(x_train)

x_val = minMaxNorm(x_val)

x_train[:4,:], y_train[:4]

(array([[0.29091901, 0.32840185],
        [0.18739645, 0.07590681],
        [0.41890954, 0.15372418],
        [0.81728824, 0.98426067]]),
 array([0., 0., 0., 0.]))

implement the activation function with backward. Output isa tuple contains the expected value with the original input. We use relu activation at here. 

# Activation

In [22]:
# To DO 
# implement the activation and its derivative
def activation(x):

    #ReLU
    out=np.maximum(0,x)
    return out, x


def activation_derivative(dvalue, x):
    
    out =np.maximum(0,x)
    p =np.where(out!=0)
    zeros =np.zeros(x.shape)
    zeros[p[0],p[1]] =1
    dx =dvalue * zeros
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return dx

# Loss function

We use softmax with cross entropy loss for the loss function. We merge the compute_loss and loss_derivative since they normally are computed together.

In [23]:
# To DO 
# implement the loss its derivative

def compute_loss(x, y):
   
    x=np.exp(x-x.max()) /np.transpose( np.array([np.sum(np.exp(x-x.max()), axis=1),]*x.shape[1]))
    target=np.zeros(x.shape)
    p=zip([i for i in range(x.shape[0])],y)
    
    target[[i for i in range(x.shape[0])],y.astype(int)]=1
    N = x.shape[0]
    ce = -np.sum(target*np.log(x+1e-9))/N
    
    grad=x
    grad[range(y.shape[0]),y.astype(int)]-=1
    grad=grad/y.shape[0]
    return ce, grad


In [24]:

def forward_pass(x,w,b):
  
  z=np.dot(x,w)+b
  cache = (x, w, b)
  
  return z, cache


In [25]:
# TO DO 
def backward_pass(dout, cache):

    x, w, b = cache

    transx =x.reshape(x.shape[0],-1)

    dx =np.dot(dout, np.transpose(w))
    dx =dx.reshape(*x.shape)
    db =np.dot(np.ones((1,x.shape[0])), dout)
    dw =np.dot(np.transpose(transx), dout)

    return dx, dw, db

#  Network

We use class to define the network, together with necessary gradient and loss function

In [26]:
class TwoLayerNet(object):

    def __init__(
        self,
        input_dim=2,
        hidden_dim=10,
        num_classes=2,
        weight_scale=1e-3,
        reg=0.0,
    ):
  
        self.params = {}
        self.reg = reg

        self.params['b1']=np.zeros((hidden_dim,))
        self.params['b2']=np.zeros((hidden_dim,))
        self.params['b3']=np.zeros((num_classes,))
        self.params['W1']=np.random.normal(0.0,weight_scale,(input_dim,hidden_dim))
        self.params['W2']=np.random.normal(0.0,weight_scale,(hidden_dim,hidden_dim))
        self.params['W3']=np.random.normal(0.0,weight_scale,(hidden_dim,num_classes))

    def predict(self,X):
        
        h1,h1_cache =forward_pass(X,self.params['W1'],self.params['b1'])
        r1,relu1_cache =activation(h1)
        h2,h2_cache =forward_pass(r1,self.params['W2'],self.params['b2'])
        r2,relu2_cache =activation(h2)
        h3,h3_cache =forward_pass(r1,self.params['W3'],self.params['b3'])
        r3,relu3_cache =activation(h3)

        return r3
        
    def loss(self, X, y=None):
      
        scores = None

        h1,h1_cache=forward_pass(X,self.params['W1'],self.params['b1'])
        r1,relu1_cache=activation(h1)
        h2,h2_cache=forward_pass(r1,self.params['W2'],self.params['b2'])
        r2,relu2_cache=activation(h2)
        h3,h3_cache=forward_pass(r2,self.params['W3'],self.params['b3'])
        r3,relu3_cache=activation(h3)

        if y is None:
            return scores

        loss, grads = 0, {}
        
        loss, grad= compute_loss(relu3_cache,y)
        loss += 0.5*np.sum(np.square(self.params['W1']))
        loss += 0.5*np.sum(np.square(self.params['W2']))
        loss += 0.5*np.sum(np.square(self.params['W3']))

        # grad+=0.5*np.sum(self.params['W1'])
        # grad+=0.5*np.sum(self.params['W2'])
        
        # print(grad.shape)
        # print(relu3_cache.shape)
        relu3_grad =activation_derivative(grad,relu3_cache)  
            
        x3_grad,grads['W3'],grads['b3'] =backward_pass(relu3_grad,h3_cache)

        relu2_grad =activation_derivative( x3_grad,relu2_cache)
        x2_grad,grads['W2'],grads['b2'] =backward_pass(relu2_grad,h2_cache)
        
        relu1_grad =activation_derivative(x2_grad,relu1_cache)
        x1_grad,grads['W1'],grads['b1'] =backward_pass(relu1_grad,h1_cache)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                             END OF YOUR CODE                             #
        ############################################################################

        return loss, grads

# Batches

In [27]:
# TO DO 
# organize the training data into batches
def make_batches(x_train, y_train, batch_size, iter):

    if (iter+1) * batch_size > x_train.shape[0]:
        return x_train[iter*batch_size:,:], y_train[iter*batch_size:]

    return x_train[iter*batch_size:(iter+1)*batch_size,:], y_train[iter*batch_size:(iter+1)*batch_size]

In [28]:
#TO DO

def update_weights(network, grads, lr):

  for index in grads.keys():
    network.params[index] -= lr*grads[index][0]
  return

In [29]:
# # To Do
# # define you stopping criteria for training
# def stopping_criteria():
#   stop = False
#   return stop

In [30]:
# TO DO
# Define the hyperparameters. Add any additional hyperparameters you might need
lr = 0.01  # learning rate
batch_size = 10
num_epochs = 100

In [31]:
def train(x_train, y_train, lr, batch_size, num_epochs, network):
  
  num_iter = x_train.shape[0]//batch_size
  # network = TwoLayerNet()

  for i in range(num_epochs):

    # print('epoch: '+str(i))

    if i % 20 == 0:
      lr=lr*0.2

    for iter in range(num_iter):

        # TO DO 
        # get a batch from the data
        x_batch, y_batch = make_batches(x_train, y_train, batch_size,iter)
        loss, grads = network.loss(x_batch,y_batch)
        # # forward the batch through the network layers
        # predicted_op = forward_pass(x_batch)

        # # compute the loss
        # loss = compute_loss(predicted_op, y_batch)

        # # perform backward pass
        # grads = backward_pass(loss)

        # update the weights of the network
        update_weights(network, grads, lr)


# Experiment

In [32]:
network = TwoLayerNet(weight_scale=0.001)

train(x_train, y_train, lr, batch_size, num_epochs, network)

num_iter = x_val.shape[0] // batch_size

print(num_iter)

num = 0

# df = pd.DataFrame({
#     "y_pred": [ np.argmax(  network.predict(x_val[i])  ) for i in range(num_iter)],
#     "y_true": y_val
# })

# (df.y_pred == df.y_true).value_counts()

for iter in range(num_iter):
    
    res = network.predict(x_val[iter])
    
    if y_val[iter] == np.argmax(res):
        # print(res)
        num += 1

print(num / num_iter)


8
0.5


# Confusion Matrix

In [33]:
from sklearn.metrics import confusion_matrix


confusion_matrix(y_true, y_pred)

ModuleNotFoundError: No module named 'sklearn'

0